In [2]:
# Import Statements
import mysql.connector
from mysql.connector import errorcode
import time

In [3]:
# Try to make connection to DB
try: 
    OBP = mysql.connector.connect(
      host="localhost",
      user="root",
      password="DogFishWhale112358.", 
      database = "off_the_beaten_path")
except mysql.connector.Error as err:
    print('Cannot connect to database: ', err)

In [4]:
#List all of Inventory Table
def list_Invetory():
    OBP_cursor = OBP.cursor()
    queryInventory = ('''
             Select * FROM Inventory
             ''')
    
    OBP_cursor.execute(queryInventory)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()    

In [5]:
#List all of Products Table
def list_Products():
    OBP_cursor = OBP.cursor()
    queryProducts = ('''
             Select * FROM Products
             ''')
    
    OBP_cursor.execute(queryProducts)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()  

In [6]:
#List all of Users Table
def list_Users():
    OBP_cursor = OBP.cursor()
    queryUsers = ('''
             Select * FROM Users
             ''')
    
    OBP_cursor.execute(queryUsers)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()    

In [7]:
#List all of OrderItems Table
def list_OrderItems():
    OBP_cursor = OBP.cursor()
    queryOrderItems = ('''
             Select * FROM OrderItems
             ''')
    
    OBP_cursor.execute(queryOrderItems)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()

In [8]:
#List all of CustOrders Table
def list_CustOrders():
    OBP_cursor = OBP.cursor()
    queryCustOrders = ('''
             Select * FROM CustOrders
             ''')
    
    OBP_cursor.execute(queryCustOrders)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()

In [9]:
#List all of Reviews Table
def list_Reviews():
    OBP_cursor = OBP.cursor()
    queryReviews = ('''
             Select * FROM Reviews
             ''')
    
    OBP_cursor.execute(queryReviews)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()

In [10]:
#List all of Shipment Table
def list_Shipment():
    OBP_cursor = OBP.cursor()
    queryShipment = ('''
             Select * FROM Shipment
             ''')
    
    OBP_cursor.execute(queryShipment)
    for row in OBP_cursor.fetchall():
        print(row)

    OBP_cursor.close()

In [11]:
# Table Print SubMenu
def solo_tableQuery():
    print('1 - Products table')
    print('2 - Inventory table')
    print('3 - Customer Orders table')
    print('4 - Order Items table')
    print('5 - Users table')
    print('6 - Reviews table')
    print('7 - Shipments table')
    while True: 
        try:
            choice = int(input("Enter your choice: "))
            if choice in range(1, 8):  # Check for valid range (1 to 7 inclusive)
                #choosing which table
                if choice == 1:
                    list_Products()
                elif choice == 2:
                    list_Invetory()
                elif choice == 3:
                    list_CustOrders()
                elif choice == 4:
                    list_OrderItems()
                elif choice == 5:
                    list_Users()
                elif choice == 6:
                    list_Reviews()
                elif choice == 7:
                    list_Shipment()

                break  # Exit the loop after a valid choice is made
            else:
                print("Invalid choice. Please enter a number between 1 and 7.")
        except ValueError as ve:
            print(f"Invalid input. Please enter numerical values. Error: {ve}")

In [12]:
#1. List which products we currently have in inventory
def list_Products_in_Invetory():
    OBP_cursor = OBP.cursor()
    try:
        queryProductsInventory = ('''
            SELECT 
                p.ProductID,
                p.Category,
                p.Brand,
                p.ProductName,
                p.Price,
                p.Color ,
                i.OnHandQty,
                i.OnOrderQty
            FROM 
                Products p
            LEFT JOIN 
                Inventory i ON p.ProductID = i.ProductID
            WHERE 
                i.OnHandQty >= 0 OR i.OnHandQty IS NULL;
                ''')
        
        OBP_cursor.execute(queryProductsInventory)
        for row in OBP_cursor.fetchall():
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        OBP.rollback()
    finally:
        OBP_cursor.close()    

In [13]:
#2. Create new products
def create_Product():
   OBP_cursor = OBP.cursor()

   print('To enter a new product you first need to provide the product specifications.')
   time.sleep(1.5)

   # questions to take in New product's specs
   allowed_categories = {'Apparel', 'Camp & Hike', 'Climb', 
											 'Run', 'Snow', 'Travel'}
   while True:
        print('Category of new product? ')
        category = input()
        if category in allowed_categories:
            break
        else:
            print(f"Invalid category. Please choose from: {allowed_categories}")
            category = input()
  
   print('Brand name of new product? ')
   brand = input()
   print('Name of new product? ')
   productName = input()
   print('Price of new product? ')
   price = float(input())
   print('Color of new product? ')
   color = input()

   product_query = ('''
             INSERT INTO Products (Category, Brand, ProductName, Price, Color)
                VALUES (%s,%s,%s,%s,%s);
             ''')
   product_queryData = (category,brand,productName,price,color)
   
   try:
        OBP_cursor.execute(product_query, product_queryData)
        OBP.commit()
        print("Product added successfully!")
   except mysql.connector.Error as err:
        print(f"Error: {err}")
        OBP.rollback()
   finally:
        OBP_cursor.close()

In [14]:
#3. Modify the amount of a particular product that we have in inventory
def update_Inventory_quantity():
  OBP_cursor = OBP.cursor()

  #Choose to update either OnHand or OnOrder quantities
  print('Enter 1 to update "On-Hand Quantity".')
  print('Enter 2 to update "On-Order Quantity".')
  choice = int(input())

  #precondition on choice
  while (choice != 1 and choice != 2):
    print("Invalid choice. Choose the number 1 or 2.")
    choice = int(input())

  if (choice == 1): #Changes On-Hand Quantity
    try:
      print('What is the productID?')
      productID = int(input())
      print('What is the new quantity?')
      newQuantity = int(input())
      Onhand_Quantity_update = ('''
                UPDATE Inventory
                  SET OnHandQty = %s
                  WHERE ProductID = %s;
                  ''')
      Inventory_updateData = (newQuantity, productID)
      OBP_cursor.execute(Onhand_Quantity_update, Inventory_updateData)
      OBP.commit()
    except ValueError as ve:
      print(f"Invalid input. Please enter numerical values. Error: {ve}")
    except mysql.connector.Error as err:
      print(f"Error: {err}")
      OBP.rollback()
    finally:
      OBP_cursor.close()

  elif (choice == 2): #Changes On-Order Quantity
    try:
      print('What is the productID?')
      productID = int(input())
      print('What is the new quantity?')
      newQuantity = int(input())
      Onorder_Quantity_update = ('''
                UPDATE Inventory
                  SET OnOrderQty = %s
                  WHERE ProductID = %s;
                  ''')
      Inventory_updateData = (newQuantity, productID)
      OBP_cursor.execute(Onorder_Quantity_update, Inventory_updateData)
      OBP.commit()
      print("Update confirmed.")
      
    except ValueError as ve:
      print(f"Invalid input. Please enter numerical values. Error: {ve}")
    except mysql.connector.Error as err:
      print(f"Error: {err}")
      OBP.rollback()
    finally:
      OBP_cursor.close()

In [15]:
#4. Delete a product from inventory
def delete_from_Inventory():
    OBP_cursor = OBP.cursor()

    print('To delete a product from Inventory, product information is needed.')
    time.sleep(1.5)
    try:
        print('what is the product ID?')
        prodID = int(input())

        InventoryDelete = ('''
                DELETE Inventory
                    FROM Inventory
                    JOIN Products ON Inventory.ProductID = Products.ProductID
                    WHERE Products.ProductID = %s;
                ''')
        
        InventoryDeleteData = (prodID,)
        OBP_cursor.execute(InventoryDelete, InventoryDeleteData)
        OBP.commit()
        print('Deletion confirmed.')

    except ValueError as ve:
      print(f"Invalid input. Please enter numerical values. Error: {ve}")
    except mysql.connector.Error as err:
      print(f"Error: {err}")
      OBP.rollback()
    finally:
        OBP_cursor.close()

In [16]:
#5. Get a list of the most popular products for a given time range
def list_PopularProducts():
    OBP_cursor = OBP.cursor()

    try:
        print('What timeframe would you like to see the most popular items from?')
        time.sleep(1.0)
        print('Starting date? (YYYY-MM-DD)') #i.e. 2022-06-01
        startDate = input()
        print('Starting date? (YYYY-MM-DD)')#i.e. 2023-12-31
        endDate = input()

        queryPopProducts = ('''
                    SELECT 
                        p.ProductID,
                        p.ProductName,
                        p.Brand,
                        p.Category,
                        COUNT(oi.OrderItemsID) AS TotalOrders,
                        SUM(oi.Quantity) AS TotalQuantitySold
                    FROM CustOrders co
                    JOIN OrderItems oi ON co.OrderItemsID = oi.OrderItemsID
                    JOIN Products p ON oi.ProductID = p.ProductID
                    WHERE co.OrderDate BETWEEN %s AND %s
                    GROUP BY 
                        p.ProductID, p.ProductName, p.Brand, p.Category
                    ORDER BY 
                        TotalQuantitySold DESC;
                ''')
        popProductsData = (startDate, endDate)
        OBP_cursor.execute(queryPopProducts, popProductsData )
        print('(ProductID, ProductName, Brand, Category, TotalOrders, TotalQuantitySold)')
        for row in OBP_cursor.fetchall():
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        OBP.rollback()
    finally:
        OBP_cursor.close()  

In [17]:
#6. Get a list of the least popular products for a given time range
def list_UnPopularProducts():
    OBP_cursor = OBP.cursor()

    try:
        print('What timeframe would you like to see the least popular items from?')
        time.sleep(1.0)
        print('Starting date? (YYYY-MM-DD)') #i.e. 2023-01-01
        startDate2 = input()
        print('Starting date? (YYYY-MM-DD)')#i.e. 2023-12-31
        endDate2 = input()

        queryUnPopProducts = ('''
                    SELECT 
                        p.ProductID,
                        p.ProductName,
                        p.Brand,
                        p.Category,
                        COUNT(oi.OrderItemsID) AS TotalOrders,
                        SUM(oi.Quantity) AS TotalQuantitySold
                    FROM CustOrders co
                    JOIN OrderItems oi ON co.OrderItemsID = oi.OrderItemsID
                    JOIN Products p ON oi.ProductID = p.ProductID
                    WHERE co.OrderDate BETWEEN %s AND %s
                    GROUP BY 
                        p.ProductID, p.ProductName, p.Brand, p.Category
                    ORDER BY 
                        TotalQuantitySold ASC;
                ''')
        unPopProductsData = (startDate2, endDate2)
        OBP_cursor.execute(queryUnPopProducts, unPopProductsData )
        print('(ProductID, ProductName, Brand, Category, TotalOrders, TotalQuantitySold)')        
        for row in OBP_cursor.fetchall():
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        OBP.rollback()
    finally:
        OBP_cursor.close()

In [18]:
# 7. Get a list of users who haven't purchased something in a few months to send promotional emails to
#   This should also include products that these users normally purchase

def list_ForPromotionalEmails():
    OBP_cursor = OBP.cursor()

    try:
        
        PromotionalEmailList = ('''
                    SELECT 
                        u.UserID,
                        u.FirstName,
                        u.LastName,
                        u.EMAIL,
                        p.ProductID,
                        p.ProductName,
                        p.Brand,
                        p.Category,
                        user_purchases.OrderDate AS LastOrderDate
                    FROM Users u
                    JOIN (
                        SELECT DISTINCT UserID
                        FROM CustOrders
                        WHERE OrderDate < DATE_SUB(NOW(), INTERVAL 3 MONTH)
                            ) inactive_users ON u.UserID = inactive_users.UserID
                    LEFT JOIN (
                        SELECT co.UserID, oi.ProductID, co.OrderDate
                        FROM CustOrders co
                        JOIN OrderItems oi ON co.OrderItemsID = oi.OrderItemsID
                            ) user_purchases ON u.UserID = user_purchases.UserID
                        JOIN Products p ON user_purchases.ProductID = p.ProductID
                    ORDER BY u.UserID, p.ProductID;
                ''')
        
        OBP_cursor.execute(PromotionalEmailList,)
        print('(UserID, FirstName , LastName       , Email                         , ProductID, ProductName         , Brand          , Category   , LastOrderDate)')        
        for row in OBP_cursor.fetchall():
            print(f'({row[0]:<6}, {row[1]:<10}, {row[2]:<15}, {row[3]:<30}, {row[4]:<9}, {row[5]:<20}, {row[6]:<15}, {row[7]:<11}, {row[8]})')
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        OBP.rollback()
    finally:
        OBP_cursor.close()

In [ ]:
# Stored Procedure 1, Search products by category
def search_By_Category():
    OBP_cursor = OBP.cursor()

    try:
        print('Which category: Apparel, Camp & Hike, Climb, Run, Snow, Travel.')
        category_name = input("Type the category name: ")
        OBP_cursor.call('search_products_by_category', (category_name,))

        # Fetch results (all products in that category)
        all_products = OBP_cursor.fetchall()
        
    for row in all_products:
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        OBP.rollback()
    finally:
        OBP_cursor.close()


In [19]:
# User Interface Menu
def menu_UI():
    print('Welcome to Off the Beaten Path outdoors equiptment database')
    print('Please select an operation you would like.')
    print('1 - Provide a list of the current products in inventory.')
    print('2 - Create a new product.')
    print('3 - Modify the quantity of a current product in inventory.')
    print('4 - Delete a product from inventory.')
    print('5 - Provide a list of the most popular products for a given time range.')
    print('6 - Provide a list of the least popular products for a given time range.')
    print('7 - Provide a list of users to send promotional emails to.')
    print('8 - Select one of seven individual tables to print.')
    print('0 - Exit menu.')

In [ ]:
#Stored Procedures SubMenu


In [22]:
# Choice Manager, links choice to functions
def handle_choice(choice):
    if choice == 1:
        list_Products_in_Invetory()
    elif choice == 2:
        create_Product()
    elif choice == 3:
        update_Inventory_quantity()
    elif choice == 4:
        delete_from_Inventory()
    elif choice == 5:
        list_PopularProducts()
    elif choice == 6:
        list_UnPopularProducts()
    elif choice == 7:
        list_ForPromotionalEmails()
    elif choice == 8:
        solo_tableQuery()
    elif choice == 0:
        print("Exiting Program ", end='')
        time.sleep(1)
        print(". ", end='')
        time.sleep(1)
        print(". ", end='')
        time.sleep(1)
        print(". ", end='')
        time.sleep(1)
        print("done.")
    else:
        print("Invalid choice. Please enter a number between 0 and 7.")

In [23]:
# Primary User Interface
menu_UI()

while True:  # Infinite loop for continuous menu usage
    try:
        choice = int(input("Enter your choice: "))
        if choice in range(0, 8):  # Check for valid range (0 to 7 inclusive)
            handle_choice(choice)
            break  # Exit the loop after a valid choice is made
        else:
            print("Invalid choice. Please enter a number between 0 and 7.")
    except ValueError as ve:
        print(f"Invalid input. Please enter numerical values. Error: {ve}")

Welcome to Off the Beaten Path outdoors equiptment database
Please select an operation you would like.
1 - Provide a list of the current products in inventory.
2 - Create a new product.
3 - Modify the quantity of a current product in inventory.
4 - Delete a product from inventory.
5 - Provide a list of the most popular products for a given time range.
6 - Provide a list of the least popular products for a given time range.
7 - Provide a list of users to send promotional emails to.
8 - Select one of seven individual tables to print.
0 - Exit menu.
Invalid input. Please enter numerical values. Error: invalid literal for int() with base 10: ''
Exiting menu . . . done.
